In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:

model = Network('model')
# Create models
node = np.empty(4, dtype = object)
node[0] = ClassSwitch(model, 'CS', [[0, 1], [1, 0]])
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.PS)
node[3] = Queue(model, 'Delay', SchedStrategy.INF)

# Create job classes
jobclass = np.empty(2, dtype = object)
jobclass[0] = ClosedClass(model, 'Class1', 15, node[3], 0)
jobclass[1] = ClosedClass(model, 'Class2', 5, node[3], 0)

# Set service times
node[1].setService(jobclass[0], Exp.fitMean(1.5))  #mean = 1.5
node[1].setService(jobclass[1], Erlang.fitMeanAndOrder(1.5, 2))  #mean = 1.5

node[2].setService(jobclass[0], Erlang.fitMeanAndOrder(1.5, 2))  #mean = 1.5
node[2].setService(jobclass[1], Exp.fitMean(1.5))  #mean = 1.5

node[3].setService(jobclass[0], Exp.fitMean(1.0))  #mean = 1
node[3].setService(jobclass[1], Exp.fitMean(1.0))  #mean = 1

# Add links
model.addLink(node[1], node[0])
model.addLink(node[2], node[0])
model.addLink(node[0], node[3])
model.addLink(node[3], node[1])
model.addLink(node[3], node[2])

# Set routing strategies
node[0].setRouting(jobclass[0], RoutingStrategy.RAND)
node[1].setRouting(jobclass[0], RoutingStrategy.RAND)
node[2].setRouting(jobclass[0], RoutingStrategy.RAND)
node[3].setRouting(jobclass[0], RoutingStrategy.RROBIN)

node[0].setRouting(jobclass[1], RoutingStrategy.RAND)
node[1].setRouting(jobclass[1], RoutingStrategy.RAND)
node[2].setRouting(jobclass[1], RoutingStrategy.RAND)

# Set weighted round robin routing using probability routing
node[3].setProbRouting(jobclass[1], node[1], 1.0/3.0)  # weight 1 out of 3 total
node[3].setProbRouting(jobclass[1], node[2], 2.0/3.0)  # weight 2 out of 3 total

In [3]:
# Solve with SolverJMT
solver = np.array([], dtype = object)
solver = np.append(solver, SolverJMT(model, 'seed', 23000, 'verbose', True))

AvgTable = np.empty(len(solver), dtype = object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()


SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/7506848299018704886/jmodel.jsim
  Station JobClass    QLen    Util    RespT  ResidT    ArvR    Tput
0  Queue1   Class1  0.9601  0.4161   3.7168  0.9292  0.2871  0.2865
1  Queue1   Class2  0.7494  0.2795   4.1231  0.6872  0.1907  0.1916
2  Queue2   Class1  7.1108  0.4300  24.7030  6.1758  0.2834  0.2826
3  Queue2   Class2  9.9742  0.5701  26.3746  8.7915  0.3815  0.3784
4   Delay   Class1  0.5610  0.5610   0.9873  0.4937  0.5668  0.5676
5   Delay   Class2  0.5668  0.5668   1.0069  0.5034  0.5617  0.5617
